# PROJET SEATTLE ENERGY BENCHMARKING
## Notebook 00 : initialisation de l'environnement et ingestion des données

---

### Identité du document
* **Statut :** Phase 1 (exploration & prototypage)
* **Date de création :** 26 Décembre 2025
* **Dernière mise à jour :** 29 Décembre 2025
* **Dépendances notebooks**: Aucune

### Description
Ce notebook établit les fondations du projet. Il assure la mise en place de l'infrastructure et la configuration des outils de gestion . L'objectif est de garantir une base reproductible avant d'entamer l'audit de qualité.

### Objectifs principaux
1. Initialiser le système de gestion de configuration via Hydra.
2. Déployer l'arborescence des répertoires du projet.
3. Récupérer et charger le jeu de données source (Immuable).
4. Établir un premier diagnostic structurel des données chargées.

### Dépendances critiques
* `hydra` : Gestion de la configuration.
* `src.utils` & `src.data` : Modules internes de support.

### LIVRABLES
1. Environnement technique initialisé (Hydra, Dossiers).
2. Dataset brut dans `data/raw/`.
3. Dictionnaire de données initial (`reports/data_dictionary.md`).
4. Note de synthèse structurelle.
5. Notebook propre

---

# Table des Matières
- [Section 0 : Importation des packages ](#section-1--configuration)
- [Section 1 : Configuration et initialisation ](#section-1--configuration)
- [Section 2 : Contextualisation & provenance](#section-2--contextualisation--provenance)
- [Section 3 : Chargement et inspection structurelle](#section-3--ingestion-et-premier-contact)
- [Section 4 : Typologie des variables](#section-4--typologie-et-cartographie-des-variables)
- [Section 5 : Focus sur la variable cible](#section-5--analyse-de-la-variable-cible)
- [Section 6 : Identification des redondances apparentes](#section-6--étude-des-redondances-et-unités)
- [Section 7 :Premières incohérences détectées](#section-7--détection-des-signaux-dalerte-immédiats)
- [Section 8 : Synthèse et génération du dictionnaire](#section-8--synthèse-et-génération-du-dictionnaire)
---

# Section 0 : Importation des packages

In [4]:
import logging
import pandas as pd
from pathlib import Path


# Import des fonctions utilitaires 

import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
SRC_PATH = PROJECT_ROOT / "src"

if str(SRC_PATH) not in sys.path:
    sys.path.insert(0, str(SRC_PATH))


from data.load_data import load_data_raw
from utils.config_loader import load_config, create_directories
from utils.eda_logger import setup_eda_logger
 


# Section 1:Configuration et initialisation

In [5]:
# Chargement de la configuration principale (Hydra)
cfg = load_config()

# Initialisation du logger
setup_eda_logger(cfg)
logger = logging.getLogger(__name__)

# Création des dossiers nécessaires à l'exécution (raw, interim, processed, reports, etc.)
create_directories(cfg)


2025-12-30 20:34:03,872 - utils.config_loader - INFO - Configuration 'config' chargée (project_root=C:\Users\HP\Desktop\temp\TODO\SEMESTRE_1\ML1\ML-prediction-CO2)
2025-12-30 20:34:03,875 - utils.config_loader - INFO - Répertoire prêt : C:\Users\HP\Desktop\temp\TODO\SEMESTRE_1\ML1\ML-prediction-CO2\data\raw
2025-12-30 20:34:03,876 - utils.config_loader - INFO - Répertoire prêt : C:\Users\HP\Desktop\temp\TODO\SEMESTRE_1\ML1\ML-prediction-CO2\data\interim
2025-12-30 20:34:03,877 - utils.config_loader - INFO - Répertoire prêt : C:\Users\HP\Desktop\temp\TODO\SEMESTRE_1\ML1\ML-prediction-CO2\data\processed
2025-12-30 20:34:03,877 - utils.config_loader - INFO - Répertoire prêt : C:\Users\HP\Desktop\temp\TODO\SEMESTRE_1\ML1\ML-prediction-CO2\figures
2025-12-30 20:34:03,877 - utils.config_loader - INFO - Répertoire prêt : C:\Users\HP\Desktop\temp\TODO\SEMESTRE_1\ML1\ML-prediction-CO2\reports


# Section 2 : Chargement et inspection structurelle

Cette section se concentre sur le chargement initial des données à partir du fichier CSV et une inspection structurelle de base. L'objectif est d'obtenir une première vue d'ensemble du dataset, de vérifier son intégrité et d'identifier tout problème évident au niveau du format ou de la structure.

## Importation du CSV

In [6]:
# Chargement des données brutes
df_raw = load_data_raw(cfg)

# Validation dimensions
n_rows, n_cols = df_raw.shape
logger.info(f"Dataset chargé ({n_rows} lignes, {n_cols} colonnes)")



2025-12-30 20:43:27,072 - data.load_data - INFO - DataFrame chargé : 3376 lignes, 46 colonnes
2025-12-30 20:43:27,102 - data.load_data - INFO -  Intégrité des données validée (Aucune modification détectée).
2025-12-30 20:43:27,106 - __main__ - INFO - Dataset chargé (3376 lignes, 46 colonnes)


---
Le chargement nous donne un DataFrame avec 3376 lignes et 46 colonnes.

## Affichage pour première impression visuelle

In [ ]:
# Forcer l'affichage de toutes les colonnes 
pd.set_option("display.max_columns", None)
# Affichage des premières lignes
print("Premières 10 lignes :")
display(df_raw.head(10))

# Affichage des dernières lignes
print("Dernières 10 lignes :")
display(df_raw.tail(10))

# Affichage d'un échantillon aléatoire
print("Échantillon aléatoire de 20 lignes :")
display(df_raw.sample(20))

,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,Address,City,State,ZipCode,TaxParcelIdentificationNumber,CouncilDistrictCode,Neighborhood,Latitude,Longitude,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),ListOfAllPropertyUseTypes,LargestPropertyUseType,LargestPropertyUseTypeGFA,SecondLargestPropertyUseType,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,YearsENERGYSTARCertified,ENERGYSTARScore,SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),DefaultData,Comments,ComplianceStatus,Outlier,TotalGHGEmissions,GHGEmissionsIntensity
0,1,2016,NonResidential,Hotel,Mayflower park hotel,405 Olive way,Seattle,WA,98101.0,0659000030,7,DOWNTOWN,47.61220,-122.33799,1927,1.0,12,88434,0,88434,Hotel,Hotel,88434.0,NaN,NaN,NaN,NaN,NaN,60.0,81.699997,84.300003,182.500000,189.000000,7226362.5,7456910.0,2.003882e+06,1.156514e+06,3946027.0,12764.529300,1276453.0,False,NaN,Compliant,NaN,249.98,2.83
1,2,2016,NonResidential,Hotel,Paramount Hotel,724 Pine street,Seattle,WA,98101.0,0659000220,7,DOWNTOWN,47.61317,-122.33393,1996,1.0,11,103566,15064,88502,"Hotel, Parking, Restaurant",Hotel,83880.0,Parking,15064.0,Restaurant,4622.0,NaN,61.0,94.800003,97.900002,176.100006,179.399994,8387933.0,8664479.0,0.000000e+00,9.504252e+05,3242851.0,51450.816410,5145082.0,False,NaN,Compliant,NaN,295.86,2.86
2,3,2016,NonResidential,Hotel,5673-The Westin Seattle,1900 5th Avenue,Seattle,WA,98101.0,0659000475,7,DOWNTOWN,47.61393,-122.33810,1969,1.0,41,956110,196718,759392,Hotel,Hotel,756493.0,NaN,NaN,NaN,NaN,NaN,43.0,96.000000,97.699997,241.899994,244.100006,72587024.0,73937112.0,2.156655e+07,1.451544e+07,49526664.0,14938.000000,1493800.0,False,NaN,Compliant,NaN,2089.28,2.19
3,5,2016,NonResidential,Hotel,HOTEL MAX,620 STEWART ST,Seattle,WA,98101.0,0659000640,7,DOWNTOWN,47.61412,-122.33664,1926,1.0,10,61320,0,61320,Hotel,Hotel,61320.0,NaN,NaN,NaN,NaN,NaN,56.0,110.800003,113.300003,216.199997,224.000000,6794584.0,6946800.5,2.214446e+06,8.115253e+05,2768924.0,18112.130860,1811213.0,False,NaN,Compliant,NaN,286.43,4.67
4,8,2016,NonResidential,Hotel,WARWICK SEATTLE HOTEL (ID8),401 LENORA ST,Seattle,WA,98121.0,0659000970,7,DOWNTOWN,47.61375,-122.34047,1980,1.0,18,175580,62000,113580,"Hotel, Parking, Swimming Pool",Hotel,123445.0,Parking,68009.0,Swimming Pool,0.0,NaN,75.0,114.800003,118.699997,211.399994,215.600006,14172606.0,14656503.0,0.000000e+00,1.573449e+06,5368607.0,88039.984380,8803998.0,False,NaN,Compliant,NaN,505.01,2.88
5,9,2016,Nonresidential COS,Other,West Precinct,810 Virginia St,Seattle,WA,98101.0,0660000560,7,DOWNTOWN,47.61623,-122.33657,1999,1.0,2,97288,37198,60090,Police Station,Police Station,88830.0,NaN,NaN,NaN,NaN,NaN,NaN,136.100006,141.600006,316.299988,320.500000,12086616.0,12581712.0,0.000000e+00,2.160444e+06,7371434.0,47151.816410,4715182.0,False,NaN,Compliant,NaN,301.81,3.10
6,10,2016,NonResidential,Hotel,Camlin,1619 9th Avenue,Seattle,WA,98101.0,0660000825,7,DOWNTOWN,47.61390,-122.33283,1926,1.0,11,83008,0,83008,Hotel,Hotel,81352.0,NaN,NaN,NaN,NaN,NaN,27.0,70.800003,74.500000,146.600006,154.699997,5758795.0,6062767.5,0.000000e+00,8.239199e+05,2811215.0,29475.800780,2947580.0,False,NaN,Compliant,NaN,176.14,2.12
7,11,2016,NonResidential,Other,Paramount Theatre,911 Pine St,Seattle,WA,98101.0,0660000955,7,DOWNTOWN,47.61327,-122.33136,1926,1.0,8,102761,0,102761,Other - Entertainment/Public Assembly,Other - Entertainment/Public Assembly,102761.0,NaN,NaN,NaN,NaN,NaN,NaN,61.299999,68.800003,141.699997,152.300003,6298131.5,7067881.5,2.276286e+06,1.065843e+06,3636655.0,3851.890137,385189.0,False,NaN,Compliant,NaN,221.51,2.16
8,12,2016,NonResidential,Hotel,311wh-Pioneer Square,612 2nd Ave,Seattle,WA,98104.0,0939000080,7,DOWNTOWN,47.60294,-122.33263,1904,1.0,15,163984,0,163984,Hotel,Hotel,163984.0,Na

Dernières 10 lignes :


,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,Address,City,State,ZipCode,TaxParcelIdentificationNumber,CouncilDistrictCode,Neighborhood,Latitude,Longitude,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),ListOfAllPropertyUseTypes,LargestPropertyUseType,LargestPropertyUseTypeGFA,SecondLargestPropertyUseType,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,YearsENERGYSTARCertified,ENERGYSTARScore,SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),DefaultData,Comments,ComplianceStatus,Outlier,TotalGHGEmissions,GHGEmissionsIntensity
3366,50210,2016,Nonresidential COS,Office,Central West HQ / Brown Bear,1403 w howe,Seattle,WA,NaN,2425039137,7,MAGNOLIA / QUEEN ANNE,47.63572,-122.37525,1952,1.0,1,13661,0,13661,Office,Office,13661.0,NaN,NaN,NaN,NaN,NaN,75.0,36.799999,40.900002,115.500000,128.399994,5.026677e+05,5.585251e+05,0.0,147323.50000,5.026678e+05,0.000000,0.000000e+00,True,NaN,Error - Correct Default Data,NaN,3.50,0.26
3367,50212,2016,Nonresidential COS,Other,Conservatory Campus,1400 E Galer St,Seattle,WA,NaN,2925049087,3,EAST,47.63228,-122.31574,1912,1.0,1,23445,0,23445,Other - Recreation,Other - Recreation,23445.0,NaN,NaN,NaN,NaN,NaN,NaN,254.899994,286.500000,380.100006,413.200012,5.976246e+06,6.716330e+06,0.0,369539.81250,1.260870e+06,47153.757810,4.715376e+06,False,NaN,Compliant,NaN,259.22,11.06
3368,50219,2016,Nonresidential COS,Mixed Use Property,Garfield Community Center,2323 East Cherry St,Seattle,WA,NaN,7544800245,3,CENTRAL,47.60775,-122.30225,1994,1.0,1,20050,0,20050,"Fitness Center/Health Club/Gym, Office, Other ...",Other - Recreation,8108.0,Fitness Center/Health Club/Gym,7726.0,Office,3779.0,NaN,NaN,90.400002,99.400002,175.199997,184.600006,1.813404e+06,1.993137e+06,0.0,225513.79690,7.694531e+05,10439.510740,1.043951e+06,False,NaN,Compliant,NaN,60.81,3.03
3369,50220,2016,Nonresidential COS,Office,Genesee/SC SE HQ,4420 S Genesee,Seattle,WA,NaN,4154300585,2,SOUTHEAST,47.56440,-122.27813,1960,1.0,1,15398,0,15398,Office,Office,15398.0,NaN,NaN,NaN,NaN,NaN,93.0,25.200001,26.900000,64.099998,66.699997,3.878100e+05,4.141724e+05,0.0,81341.39844,2.775369e+05,1102.729980,1.102730e+05,True,NaN,Error - Correct Default Data,NaN,7.79,0.51
3370,50221,2016,Nonresidential COS,Other,High Point Community Center,6920 34th Ave SW,Seattle,WA,NaN,2524039059,1,DELRIDGE NEIGHBORHOODS,47.54067,-122.37441,1982,1.0,1,18261,0,18261,Other - Recreation,Other - Recreation,18261.0,NaN,NaN,NaN,NaN,NaN,NaN,51.000000,56.200001,126.000000,136.600006,9.320821e+05,1.025432e+06,0.0,185334.70310,6.323620e+05,2997.199951,2.997200e+05,False,NaN,Compliant,NaN,20.33,1.11
3371,50222,2016,Nonresidential COS,Office,Horticulture building,1600 S Dakota St,Seattle,WA,NaN,1624049080,2,GREATER DUWAMISH,47.56722,-122.31154,1990,1.0,1,12294,0,12294,Office,Office,12294.0,NaN,NaN,NaN,NaN,NaN,46.0,69.099998,76.699997,161.699997,176.100006,8.497457e+05,9.430032e+05,0.0,153655.00000,5.242709e+05,3254.750244,3.254750e+05,True,NaN,Error - Correct Default Data,NaN,20.94,1.70
3372,50223,2016,Nonresidential COS,Other,International district/Chinatown CC,719 8th Ave S,Seattle,WA,NaN,3558300000,2,DOWNTOWN,47.59625,-122.32283,2004,1.0,1,16000,0,16000,Other - Recreation,Other - Recreation,16000.0,NaN,NaN,NaN,NaN,NaN,NaN,59.400002,65.900002,114.199997,118.900002,9.502762e+05,1.053706e+06,0.0,116221.00000,3.965461e+05,5537.299805,5.537300e+05,False,NaN,Compliant,NaN,32.17,2.01
3373,50224,2016,Nonresidential COS,Other,Queen Anne Pool,1920 1st Ave W,Seattle,WA,NaN,1794501150,7,MAGNOLIA / QUEEN ANNE,47.63644,-122.35784,1974,1.0,1,13157,0,13157,"Fitness Center/Health Club/Gym, Other - Recrea...",Other - Recreation,7583.0,Fitness Center/Health Club/Gym,5574.0,Swimming Pool,0.0,NaN,NaN,438.200012,460.100006,744.799988,767.799

Échantillon aléatoire de 20 lignes :


,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,Address,City,State,ZipCode,TaxParcelIdentificationNumber,CouncilDistrictCode,Neighborhood,Latitude,Longitude,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),ListOfAllPropertyUseTypes,LargestPropertyUseType,LargestPropertyUseTypeGFA,SecondLargestPropertyUseType,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,YearsENERGYSTARCertified,ENERGYSTARScore,SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),DefaultData,Comments,ComplianceStatus,Outlier,TotalGHGEmissions,GHGEmissionsIntensity
2227,24868,2016,NonResidential,Worship Facility,Magnolia,2415 31st Ave,Seattle,WA,98199.0,8127700530,7,MAGNOLIA / QUEEN ANNE,47.64000,-122.39743,1965,1.0,1,20140,0,20140,"Parking, Worship Facility",Worship Facility,22000.0,Parking,0.0,NaN,NaN,NaN,35.0,45.099998,52.900002,59.000000,67.800003,9.927947e+05,1.164371e+06,0.0,3.587040e+04,122390.00,8704.049805,870405.000,False,NaN,Compliant,NaN,47.08,2.34
1119,20928,2016,NonResidential,Worship Facility,St. Paul Church & School,10001 57th Ave. S,Seattle,WA,98178.0,1686400005,2,SOUTHEAST,47.51042,-122.26277,1954,1.0,2,27876,0,27876,Worship Facility,Worship Facility,44816.0,NaN,NaN,NaN,NaN,NaN,69.0,40.799999,46.799999,71.400002,78.500000,1.829981e+06,2.095655e+06,0.0,1.792442e+05,611581.00,12184.000000,1218400.000,False,NaN,Compliant,NaN,68.97,2.47
2780,26919,2016,Multifamily LR (1-4),Low-Rise Multifamily,Lambda Chi Alpha,4509 19th Ave NE,Seattle,WA,98105.0,8823902740,4,NORTHEAST,47.66164,-122.30762,1928,1.0,3,20629,0,20629,Multifamily Housing,Multifamily Housing,20629.0,NaN,NaN,NaN,NaN,NaN,52.0,58.599998,64.199997,96.199997,102.099998,1.209434e+06,1.325284e+06,0.0,1.002621e+05,342094.00,8673.390625,867339.000,False,NaN,Compliant,NaN,48.45,2.35
1516,22194,2016,Multifamily LR (1-4),Low-Rise Multifamily,Sixty Five,1433 NW 64th,Seattle,WA,98107.0,2767600820,6,BALLARD,47.67504,-122.37529,1986,1.0,4,44957,0,44957,"Multifamily Housing, Parking",Multifamily Housing,30044.0,Parking,6000.0,NaN,NaN,NaN,71.0,30.799999,33.000000,96.800003,103.599998,9.262612e+05,9.908829e+05,0.0,2.714716e+05,926261.00,0.000000,0.000,False,NaN,Compliant,NaN,6.46,0.14
6,10,2016,NonResidential,Hotel,Camlin,1619 9th Avenue,Seattle,WA,98101.0,0660000825,7,DOWNTOWN,47.61390,-122.33283,1926,1.0,11,83008,0,83008,Hotel,Hotel,81352.0,NaN,NaN,NaN,NaN,NaN,27.0,70.800003,74.500000,146.600006,154.699997,5.758795e+06,6.062768e+06,0.0,8.239199e+05,2811215.00,29475.800780,2947580.000,False,NaN,Compliant,NaN,176.14,2.12
1913,23789,2016,Multifamily HR (10+),High-Rise Multifamily,Highlander Condominiums,525 Belmount Ave East,Seattle,WA,98102.0,3302700000,3,EAST,47.62395,-122.32442,1965,1.0,11,66150,11290,54860,Multifamily Housing,Multifamily Housing,66150.0,NaN,NaN,NaN,NaN,NaN,21.0,37.700001,40.799999,118.500000,128.000000,2.496808e+06,2.696894e+06,0.0,7.317724e+05,2496807.00,0.000000,0.000,False,NaN,Compliant,NaN,17.41,0.26
710,19682,2016,Multifamily MR (5-9),Mid-Rise Multifamily,The Audrey at Belltown,2922 Western Avenue,Seattle,WA,98121.0,0695000005,7,DOWNTOWN,47.61687,-122.35376,1992,1.0,8,188717,53284,135433,"Multifamily Housing, Parking",Multifamily Housing,135433.0,Parking,53284.0,NaN,NaN,NaN,85.0,31.000000,32.400002,90.099998,93.400002,4.192895e+06,4.381679e+06,0.0,1.093983e+06,3732669.00,4602.260254,460226.000,False,NaN,Compliant,NaN,50.46,0.27
3367,50212,2016,Nonresidential COS,Other,Conservatory Campus,1400 E Galer St,Seattle,WA,NaN,2925049087,3,EAST,47.63228,-122.31574,1912,1.0,1,23445,0,23445,Other - Recreation,Other - Recreation,23445.0,NaN,NaN,NaN,NaN,NaN,NaN,254.899994,286.500000,380.100006,413.200012,5.976246e+06,6.716330e+06,0.0,3.695398e+05,1260869.84,47153.757810,4715375.781,False,NaN,Compliant,NaN,259.22,11.06
1015,20512,